In [ ]:
import gempy as gp
import numpy as np
import matplotlib.pyplot as plt
import os
from pandas import *

In [ ]:
import pandas as pd
data_path = '/Users/juliano/Google Drive/Master/Gempy/Model/Results/'

data = pd.read_csv(os.path.join(data_path,"FZ_66_points_duplicate.csv"))
data2 = pd.read_csv(os.path.join(data_path, "FZ_66_orientations_duplicate.csv"))

In [ ]:
data

In [ ]:
print(data.dtypes)
print(data2.dtypes)

In [ ]:
data_path = '/Users/juliano/Google Drive/Master/Gempy/Model/Results/'

geo_model = gp.create_model('FZ_model')
gp.init_data(geo_model, extent=[679330, 679530, 151805, 152005, 1390, 1590],
             resolution=[50, 50, 50],
             surface_points_df = data,
             orientations_df = data2)

In [ ]:
#geo_model.surfaces.colors.change_colors({'39':'#fdfefe'})
gp.map_stack_to_surfaces(geo_model,
                         {"Fault_66_B": 'S15',
                          "Fault_66_C": 'S21', 
                          "Strat_Series": ('basement')},
                         remove_unused_series=True)

In [ ]:
geo_model.set_is_fault([  "Fault_66_B",
                          "Fault_66_C"], change_color=False)

In [ ]:
geo_model.faults.faults_relations_df

In [ ]:
gpv = gp.plot_3d(geo_model, image=False, plotter_type='basic')

In [ ]:
section_dict = {'section1': ([679530, 151805], [679330, 152005], [100, 100])}  # p1,p2,resolution
geo_model.set_section_grid(section_dict)

##Definiere entlang welcher Richtung die Section geplottet werden soll

In [ ]:
gp.plot.plot_section_traces(geo_model)
plt.show()

In [ ]:
gp.set_interpolator(geo_model,
                    compile_theano=True,
                    theano_optimizer='fast_compile',
                    )

In [ ]:
gp.get_data(geo_model, 'kriging')

In [ ]:
sol = gp.compute_model(geo_model)

In [ ]:
geo_model.solutions.scalar_field_at_surface_points

In [ ]:
gp.plot_2d(geo_model, section_names=['section1'], legend=False, show_lith=False)
#plt.show()
plt.savefig('FEAR_Site1_largeFZ_Jordan_CrossSecAlongTunnel.png')

In [ ]:
gp.plot_2d(geo_model, cell_number=25, direction='z', legend=False, show_lith=False)
#plt.show()
plt.savefig('FEAR_Site1_largeFZ_Jordan_PlanAtTunnel.png')

In [ ]:
ver, sim = gp.get_surfaces(geo_model)
gpv = gp.plot_3d(geo_model, image=False, plotter_type='basic')

## Information Entropy

In [ ]:
# execute this cell a couple of times and see how location of fault in section changes

mask_surfpoints = geo_model.surface_points.df.surface.isin(['fault']) # perturb only fault
indexes = geo_model.surface_points.df[mask_surfpoints].index

rand_val = np.random.uniform(-100, 100)
we = geo_model.surface_points.df['X'].values[mask_surfpoints] + rand_val
ns = geo_model.surface_points.df['Y'].values[mask_surfpoints] + rand_val
d = geo_model.surface_points.df['Z'][mask_surfpoints] + rand_val
geo_model.modify_surface_points(indexes, X=we, Y=ns, Z=d)

geo_model.update_to_interpolator()

_=gp.compute_model(geo_model, compute_mesh=False)

#_=gp.plot.plot_section_by_name(geo_model, 's1', show_data=True, radius=50)

In [ ]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
# unique lith ids
lith_id = np.unique(np.round(geo_model.solutions.sections[0]).astype(int))

# setup solution arrays
geomap  = np.round(geo_model.solutions.geological_map[0]).astype(int).ravel()
section1  = np.round(geo_model.solutions.sections[0]).astype(int).ravel()
block = np.round(geo_model.solutions.lith_block).astype(int)

#init counters
count_map = np.zeros((len(lith_id), geomap.shape[0]))
count_section = np.zeros((len(lith_id), section1.shape[0]))
count_block = np.zeros((len(lith_id), block.shape[0]))

In [ ]:
mask_surfpoints = geo_model.surface_points.df.surface.isin(['fault'])
indexes = geo_model.surface_points.df[mask_surfpoints].index

In [ ]:
# copy initial dataframes to reset data before every iteration
import copy

west_east = copy.copy(geo_model.surface_points.df['X'])
north_south = copy.copy(geo_model.surface_points.df['Y'])
depth = copy.copy(geo_model.surface_points.df['Z'])

surfindexes = list(geo_model.surface_points.df.index)

In [ ]:
n_iter=13 #number of iterations

for i in tqdm(range(n_iter)):
    plt.cla()
    plt.clf()
    
    # make virgin again
    geo_model.modify_surface_points(surfindexes, X=west_east, Y=north_south, Z=depth)
    geo_model.update_to_interpolator()
    
    # perturb data
    rand_val = np.random.uniform(-100, 100)
    we = west_east[mask_surfpoints] + rand_val
    geo_model.modify_surface_points(indexes, X=we)

    geo_model.update_to_interpolator()

    _=gp.compute_model(geo_model, compute_mesh=False)
    
    
    ##### calculate and update probability fields #####
    geomap  = np.round(geo_model.solutions.geological_map[0]).astype(int)[0]
    section1  = np.round(geo_model.solutions.sections[0]).astype(int)[0]
    block = np.round(geo_model.solutions.lith_block).astype(int)
    
    
    for i, l_id in enumerate(lith_id): #enumerate through all liths
        count_map[i][geomap == l_id] += 1 #sum up frequency
        count_section[i][section1 == l_id] += 1 
        count_block[i][block == l_id] +=1 #block is raveled so no need for indexing

#### finish probability calculation and save as numpy arrays ####
prob_map = count_map/n_iter
prob_section = count_section/n_iter
prob_block = count_block/n_iter